# Segmenting and Clustering Neighborhoods in Toronto, Canada

### Data Wrangling

Importing 'pandas' and Creating a pandas dataframe from Wikipedia table

In [1]:
import pandas as pd
zip_canada = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_zip_can = pd.read_html(zip_canada)[0]

Dropping rows with 'Not assigned' Borough

In [2]:
df_zip_can = df_zip_can[df_zip_can['Borough'] != 'Not assigned']

Sorting the 'Postal Code' and Resetting the index

In [3]:
df_zip_can.sort_values('Postal Code', axis=0, ascending=True, inplace=True)
df_zip_can.reset_index(drop= True, inplace=True)

In [4]:
print('Number of rows =' ,df_zip_can.shape[0])

Number of rows = 103


In [5]:
df_zip_can.head()

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
post_can = 'https://cocl.us/Geospatial_data'
df_post_can = pd.read_csv(post_can)

In [7]:
df_post_can.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Adding 'Latitude' & 'Longitude' columns to the DataFrame

In [8]:
df_zip_can['Latitude'] = df_post_can['Latitude']
df_zip_can['Longitude'] = df_post_can['Longitude']

In [9]:
df_zip_can.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


Checking for unique Borough names

In [10]:
df_zip_can['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       'Mississauga', 'Etobicoke'], dtype=object)

Creating a new DataFrame containing only the Neighborhoods in Toronto

In [11]:
df_toronto = df_zip_can[df_zip_can['Borough'].isin(['Central Toronto','Downtown Toronto','East Toronto','West Toronto'])]
df_toronto.reset_index(drop=True, inplace=True)
df_toronto.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


Importing required libraries.

In [12]:
import numpy as np 

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans #for clustering

import folium # for map

Creating a map of Toronto Neighborhood and adding labels of borough & neighborhood

In [13]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="on_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='red',
        fill=True,
        fill_color='white',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#Please drop the notebook's link to "http://nbviewer.org/" in order to view map.
#Although I 'Trusted' the notebook, it still doesn't appear on github repository.
#I would be thankful if somebody would show me the way to do it.

### Obtaining nearby Venues

Defining Foursquare Credentials and Version

In [14]:
CLIENT_ID = '3KCOZ2WORUZ0TSBM3C1505MFUBJ5WELSZZAD1S3MIEDUTE0H' # Foursquare ID
CLIENT_SECRET = '24EGVFS541ZAHQL02B115C2EC3XARXYSWWS0QR1O1C1UHH5J' # Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
toronto_venues = getNearbyVenues(names = df_toronto['Neighbourhood'],
                  latitudes = df_toronto['Latitude'],
                  longitudes = df_toronto['Longitude'],
                  radius = 1000)

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West, Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High 

In [18]:
print(toronto_venues.shape)
toronto_venues.head()

(3196, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
2,The Beaches,43.676357,-79.293031,Ed's Real Scoop,43.672630,-79.287993,Ice Cream Shop
3,The Beaches,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery
4,The Beaches,43.676357,-79.293031,The Fox Theatre,43.672801,-79.287272,Indie Movie Theater


Grouping by Neignborhoods for nearby venues

In [19]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,100,100,100,100,100,100
"Brockton, Parkdale Village, Exhibition Place",100,100,100,100,100,100
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",50,50,50,50,50,50
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",14,14,14,14,14,14
Central Bay Street,100,100,100,100,100,100
Christie,100,100,100,100,100,100
Church and Wellesley,100,100,100,100,100,100
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,100,100,100,100,100,100


In [20]:
len(toronto_venues['Venue Category'].unique())

279

Processing onehot to obtain which venues are located in each neighborhood.

In [21]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix='', prefix_sep='')

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

neighborhood = toronto_onehot.pop('Neighborhood')
toronto_onehot.insert(0, 'Neighborhood', neighborhood)

toronto_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Quad,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Travel,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Yoga Studio,Zoo
0,T

Regrouping each Neighborhood

In [22]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Amphitheater,Animal Shelter,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Stadium,Beach,Beach Bar,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Castle,Cemetery,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Quad,College Rec Center,College Theater,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Distribution Center,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Travel,German Restaurant,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Latin American Restaurant,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Monument / Landmark,Movie Theater,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Organic Grocery,Other Great Outdoors,Other Repair Shop,Pakistani Restaurant,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pide Place,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,River,Rock Climbing Spot,Rock Club,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stationery Store,Steakhouse,Storage Facility,Street Art,Supermarket,Sushi Restaurant,Syrian Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tech Startup,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Track,Trail,Train Station,Tree,Turkish Restaurant,Udon Restaurant,University,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Yoga Studio,Zoo
0,B

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Finding top 10 venues for each neighborhood.

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Café,Japanese Restaurant,Hotel,Restaurant,Park,Beer Bar,Liquor Store,Bakery,Farmers Market
1,"Brockton, Parkdale Village, Exhibition Place",Café,Coffee Shop,Bar,Restaurant,Furniture / Home Store,Bakery,Gift Shop,Tibetan Restaurant,Vegetarian / Vegan Restaurant,Thrift / Vintage Store
2,"Business reply mail Processing Centre, South C...",Park,Coffee Shop,Pizza Place,Brewery,Italian Restaurant,Sushi Restaurant,Fast Food Restaurant,Bakery,Burrito Place,Snack Place
3,"CN Tower, King and Spadina, Railway Lands, Har...",Café,Harbor / Marina,Coffee Shop,Scenic Lookout,Airport,Sculpture Garden,Dog Run,Dance Studio,Park,Garden
4,Central Bay Street,Coffee Shop,Ramen Restaurant,Park,Sushi Restaurant,Hotel,Café,Electronics Store,Plaza,Bubble Tea Shop,Thai Restaurant


### Clustering

In [25]:
k = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=k, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_[0:10] 

array([2, 0, 1, 3, 2, 0, 1, 2, 1, 1])

Making a new dataframe and adding clusters

In [26]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Pub,Coffee Shop,Pizza Place,Beach,Breakfast Spot,Japanese Restaurant,Health Food Store,Bar,Bakery,Grocery Store
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Café,Coffee Shop,Pub,Spa,Bank,Italian Restaurant,Restaurant,Ramen Restaurant,Ice Cream Shop
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Indian Restaurant,Coffee Shop,Restaurant,Grocery Store,Park,Beach,Gym,Bakery,Butcher,Café
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Coffee Shop,Bar,American Restaurant,Diner,Vietnamese Restaurant,Café,Brewery,Bakery,French Restaurant,Italian Restaurant
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Bookstore,Coffee Shop,Trail,College Gym,College Quad,Gym / Fitness Center,Café,Falafel Restaurant,Event Space


Adding clusters to map

In [27]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)
color = ['red','blue','green','black','purple']
fill = ['white','yellow','green','yellow','purple']

x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=color[cluster-1],
        fill=True,
        fill_color=fill[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#Please drop the notebook's link to "http://nbviewer.org/" in order to view map.
#Although I 'Trusted' the notebook, it still doesn't appear on github repository.
#I would be thankful if somebody would show me the way to do it.

Cluster 1 

Area where lots of coffee shops, bar, international restaurants and bakeries are located.
Suitable for any meal.

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Pub,Coffee Shop,Pizza Place,Beach,Breakfast Spot,Japanese Restaurant,Health Food Store,Bar,Bakery,Grocery Store
2,East Toronto,0,Indian Restaurant,Coffee Shop,Restaurant,Grocery Store,Park,Beach,Gym,Bakery,Butcher,Café
3,East Toronto,0,Coffee Shop,Bar,American Restaurant,Diner,Vietnamese Restaurant,Café,Brewery,Bakery,French Restaurant,Italian Restaurant
11,Downtown Toronto,0,Café,Park,Gastropub,Japanese Restaurant,Diner,Thai Restaurant,Italian Restaurant,Garden,Farm,Restaurant
24,Central Toronto,0,Italian Restaurant,Café,Coffee Shop,Vegetarian / Vegan Restaurant,Restaurant,Gym,Pub,Museum,Grocery Store,Bakery
25,Downtown Toronto,0,Café,Bar,Bakery,Vegetarian / Vegan Restaurant,Bookstore,Coffee Shop,Beer Bar,Mexican Restaurant,Comfort Food Restaurant,Museum
26,Downtown Toronto,0,Café,Bar,Vegetarian / Vegan Restaurant,Coffee Shop,Mexican Restaurant,Yoga Studio,Art Gallery,Arts & Crafts Store,French Restaurant,Dumpling Restaurant
30,Downtown Toronto,0,Korean Restaurant,Café,Coffee Shop,Grocery Store,Cocktail Bar,Mexican Restaurant,Pizza Place,Sandwich Place,Comedy Club,Park
32,West Toronto,0,Café,Restaurant,Bar,Coffee Shop,Italian Restaurant,Bakery,Asian Restaurant,Vegetarian / Vegan Restaurant,Cocktail Bar,Pizza Place
33,West Toronto,0,Café,Coffee Shop,Bar,Restaurant,Furniture / Home Store,Bakery,Gift Shop,Tibetan Restaurant,Vegetarian / Vegan Restaurant,Thrift / Vintage Store


Cluster 2

A sutible zone for college students with bookstore, parks , gym and event space for recreation.

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,East Toronto,1,Greek Restaurant,Café,Coffee Shop,Pub,Spa,Bank,Italian Restaurant,Restaurant,Ramen Restaurant,Ice Cream Shop
5,Central Toronto,1,Coffee Shop,Italian Restaurant,Restaurant,Café,Dessert Shop,Gym,Fast Food Restaurant,Sushi Restaurant,Pizza Place,Pharmacy
6,Central Toronto,1,Italian Restaurant,Garden,Skating Rink,Sporting Goods Shop,Restaurant,Diner,Café,Coffee Shop,Mexican Restaurant,Park
7,Central Toronto,1,Coffee Shop,Italian Restaurant,Sushi Restaurant,Café,Pizza Place,Indian Restaurant,Restaurant,Dessert Shop,Gym,Mexican Restaurant
8,Central Toronto,1,Grocery Store,Coffee Shop,Italian Restaurant,Gym,Park,Thai Restaurant,Restaurant,Bagel Shop,Sushi Restaurant,Café
9,Central Toronto,1,Coffee Shop,Sushi Restaurant,Italian Restaurant,Thai Restaurant,Grocery Store,Restaurant,Park,Bagel Shop,Sandwich Place,Café
10,Downtown Toronto,1,Park,Coffee Shop,Grocery Store,Convenience Store,Bank,Pie Shop,Sandwich Place,Candy Store,Filipino Restaurant,Metro Station
12,Downtown Toronto,1,Coffee Shop,Japanese Restaurant,Gay Bar,Park,Café,Sushi Restaurant,Men's Store,Italian Restaurant,Ramen Restaurant,Pizza Place
13,Downtown Toronto,1,Coffee Shop,Pub,Theater,Park,Café,Diner,Breakfast Spot,Bakery,Restaurant,Indian Restaurant
22,Central Toronto,1,Italian Restaurant,Sushi Restaurant,Pharmacy,Coffee Shop,Café,Bank,Playground,Bagel Shop,Skating Rink,Dry Cleaner


Cluster 3

A suitable area for visitors with a number of hotels and coffee shops. Japanese and Thai restaurants can also be found. 
Theaters and concert halls are also in this area.


In [30]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Downtown Toronto,2,Coffee Shop,Gastropub,Japanese Restaurant,Diner,Restaurant,Hotel,Italian Restaurant,Electronics Store,New American Restaurant,Cosmetics Shop
15,Downtown Toronto,2,Coffee Shop,Café,Restaurant,Gastropub,Japanese Restaurant,Italian Restaurant,Bakery,Beer Bar,Hotel,Seafood Restaurant
16,Downtown Toronto,2,Coffee Shop,Café,Japanese Restaurant,Hotel,Restaurant,Park,Beer Bar,Liquor Store,Bakery,Farmers Market
17,Downtown Toronto,2,Coffee Shop,Ramen Restaurant,Park,Sushi Restaurant,Hotel,Café,Electronics Store,Plaza,Bubble Tea Shop,Thai Restaurant
18,Downtown Toronto,2,Coffee Shop,Café,Hotel,Theater,Gym,Restaurant,Pizza Place,Italian Restaurant,Japanese Restaurant,Beer Bar
19,Downtown Toronto,2,Coffee Shop,Hotel,Café,Restaurant,Park,Brewery,Japanese Restaurant,Gym,Theater,Aquarium
20,Downtown Toronto,2,Coffee Shop,Hotel,Café,Japanese Restaurant,Restaurant,Italian Restaurant,Concert Hall,Theater,Gastropub,Plaza
21,Downtown Toronto,2,Coffee Shop,Hotel,Restaurant,Café,Beer Bar,Japanese Restaurant,Gastropub,Concert Hall,Farmers Market,Thai Restaurant
28,Downtown Toronto,2,Coffee Shop,Café,Restaurant,Hotel,Japanese Restaurant,Beer Bar,Gastropub,Seafood Restaurant,Art Gallery,Farmers Market
29,Downtown Toronto,2,Coffee Shop,Hotel,Café,Restaurant,Japanese Restaurant,Theater,Italian Restaurant,Concert Hall,Beer Bar,Plaza


Cluster 4

Costal area with harbor and airport. Sutibal for an evening walk and evening chill in park or garden.

In [31]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
27,Downtown Toronto,3,Café,Harbor / Marina,Coffee Shop,Scenic Lookout,Airport,Sculpture Garden,Dog Run,Dance Studio,Park,Garden


Cluster 5

A number of banks, international restaurants and coffee shops are located in this area. 
A number of parks can also be found.

In [32]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,4,Park,Bookstore,Coffee Shop,Trail,College Gym,College Quad,Gym / Fitness Center,Café,Falafel Restaurant,Event Space
